In [10]:
import matplotlib.pyplot as plt
%matplotlib inline    
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import numpy as np
import sys, copy
from pathlib import Path
lib_dir = (Path("__file__").parent).resolve()
if str(lib_dir) not in sys.path: sys.path.insert(0, str(lib_dir))
from AdapAug.data import get_dataloaders
from theconf import Config as C, ConfigArgumentParser
from PIL import Image
from sklearn.manifold import TSNE
from matplotlib.pyplot import cm
import numpy as np
from collections import Counter
import pandas as pd
from AdapAug.controller import Controller
from AdapAug.networks.wideresnet import WideResNet

In [2]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

In [3]:
C.get()["aug"] = "default"
C.get()["cutout"] = 16
C.get()["cv_num"] = 5

In [18]:
controller = Controller(n_subpolicy=2, lstm_size=100, emb_size=32,
                            operation_prob=0).cuda()
model = WideResNet(28, 10, dropout_rate=0.0, num_classes=100).cuda()
ckpt = torch.load("/home/server36/yongsu/adapaug/AdapAug/models/cifar100_v3_ppo_np2_aw9.900e-01_rt3_lr0.0888_0/target_network.pt")
# ckpt = torch.load("/home/server36/yongsu/adapaug/AdapAug/models/cifar100_v3_ppo_np2_aw1.000e-01_rt3_lr0.0128_0/target_network.pt")
model.load_state_dict(ckpt["model"])

<All keys matched successfully>

In [4]:
_, total_loader, valid_loader, test_loader = get_dataloaders("cifar100", 128, "/home/server36/data", 0.4, split_idx=0,
                                                             controller=None,
                                                             rand_val=True, _transform="default", validation=True)

In [20]:
print(total_loader.dataset.transform.transforms)
# total_loader.dataset.transform.transforms.index(transforms.ToTensor)
for i,x in enumerate(total_loader.dataset.transform.transforms):
    if x == transforms.ToTensor:
        print(i)
        break
else:
    print("nothing")

[RandomCrop(size=(32, 32), padding=4), RandomHorizontalFlip(p=0.5), ToTensor(), Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201)), <AdapAug.data.CutoutDefault object at 0x7f43eb712eb8>]
nothing


In [19]:
ts = transforms.Compose([])
x = torch.rand(3,32,32)
print(ts(x)==x)

tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]]]

In [16]:
_, _, _, _test_loader = get_dataloaders("cifar100", 128, "/home/server36/data", 0.4, split_idx=0,
                                                             rand_val=False, _transform="default", validation=False)

In [7]:
print(test_loader.dataset.dataset.transform)
print(test_loader.dataset.dataset.policies)

Compose(
    ToTensor()
    Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
)
None


In [19]:
from AdapAug.metrics import accuracy
accs = 0.
cnt = 0
for data, label in _test_loader:
    data, label = data.cuda(), label.cuda()
    accs += accuracy(model(data), label)[0] * len(data)
    cnt += len(data)
print(accs/cnt)

tensor(0.7922, device='cuda:0')


In [ ]:
isinstance(label, list)

In [ ]:
cmap = cm.get_cmap('jet', num_classes)
plt.scatter(xs,ys,s=1.,alpha=0.2,c=total_trainset.targets, cmap=cmap)
plt.colorbar()
plt.show()

In [ ]:
fig, axes = plt.subplots(cv_num, repeat, figsize=(4*repeat,4*cv_num))
for cv_id in range(cv_num):
    for r, id_dist in enumerate(gr_dist_move[cv_id]):
        id_dist = torch.max(id_dist,-1)[1].numpy()
#         print(id_dist)
        ax = axes[cv_id][r]
        ax.scatter(xs, ys, c=id_dist, s=0.2, alpha=0.5, label=Counter(id_dist), cmap=cmap)
        print(Counter(id_dist))
        if r == 0:
            ax.set_ylabel(f"cv {cv_id}")
        if cv_id == 0:
            ax.set_title(f"repeat {r}")
#         ax.legend()
plt.show()

In [ ]:
gr_ids = gr_dist_move["last"].max(-1)[1].numpy()
print(Counter(gr_ids))
print(gr_dist_move["last"])
plt.scatter(xs, ys, s=2., alpha=0.4, c=gr_ids, cmap=cmap)

In [ ]:
cmap = cm.get_cmap('jet', 2)
plt.scatter(xs,ys,s=1.,alpha=0.2,c=[1 if x<10 else 2 for x in total_trainset.targets], cmap=cmap)
plt.show()

In [ ]:
# gr_ids
_, trainloader, validloader, testloader = get_dataloaders("cifar10_svhn", 128, "/home/server32/data/", 0.0, gr_id=0, gr_ids=gr_ids)

In [ ]:
print(len(testloader.dataset))

In [ ]:
load_path = "/home/server32/yongsu/fast-autoaugment/FastAutoAugment/models/logging_b1000/search_cifar10_svhn_wresnet40_2_group2_4_cv0_ratio0.4_bo_result.csv"
result_csv = pd.read_csv(load_path)
minus_loss = result_csv['minus_loss']
acc = result_csv["top1_valid"]

In [ ]:
maxline = [max(acc[:x]) for x in range(1,len(acc))]
plt.plot(acc, label="Acc")
plt.plot(maxline, label="Max")
plt.axvline(x=199,c='red')
plt.text(200, acc[199], "{:.4f}".format(acc[199]))
plt.text(200, maxline[198], "{:.4f}".format(maxline[198]))
plt.axvline(x=24,c='blue')
plt.text(25, acc[24], "{:.4f}".format(acc[24]))
plt.text(25, maxline[23], "{:.4f}".format(maxline[23]))
plt.text(len(acc), maxline[-1], "{:.4f}".format(maxline[-1]))
plt.legend()
plt.show()